In [1]:
import numpy as np
import cupy as cp
import time

In [2]:
def vector_sum_cpu(vector):
    result_cpu = 0
    for element in vector:
        result_cpu += element
    return result_cpu

In [3]:
def vector_sum_gpu(vector):
    vector_gpu = cp.asarray(vector)
    result_gpu = cp.sum(vector_gpu)
    result_cpu = cp.asnumpy(result_gpu)
    return result_cpu

In [15]:
vector_size = 1000
vector = np.random.rand(vector_size)

# Сложение на CPU
start_time_cpu = time.time()
result_cpu = vector_sum_cpu(vector)
end_time_cpu = time.time()

cpu_time = end_time_cpu - start_time_cpu
print(f"Сумма на CPU: {result_cpu}, Время выполнения на CPU: {cpu_time} секунд")

# Сложение на GPU
start_time_gpu = time.time()
result_gpu = vector_sum_gpu(vector)
end_time_gpu = time.time()

gpu_time = end_time_gpu - start_time_gpu
print(f"Сумма на GPU: {result_gpu}, Время выполнения на GPU: {gpu_time} секунд")    

Сумма на CPU: 502.99950407721786, Время выполнения на CPU: 0.0005352497100830078 секунд
Сумма на GPU: 502.99950407721775, Время выполнения на GPU: 0.0021071434020996094 секунд


In [16]:
vector_size = 10000
vector = np.random.rand(vector_size)

# Сложение на CPU
start_time_cpu = time.time()
result_cpu = vector_sum_cpu(vector)
end_time_cpu = time.time()

cpu_time = end_time_cpu - start_time_cpu
print(f"Сумма на CPU: {result_cpu}, Время выполнения на CPU: {cpu_time} секунд")

# Сложение на GPU
start_time_gpu = time.time()
result_gpu = vector_sum_gpu(vector)
end_time_gpu = time.time()

gpu_time = end_time_gpu - start_time_gpu
print(f"Сумма на GPU: {result_gpu}, Время выполнения на GPU: {gpu_time} секунд")    

Сумма на CPU: 5016.4290514109825, Время выполнения на CPU: 0.0010020732879638672 секунд
Сумма на GPU: 5016.429051410992, Время выполнения на GPU: 0.0009996891021728516 секунд


In [17]:
vector_size = 100000
vector = np.random.rand(vector_size)

# Сложение на CPU
start_time_cpu = time.time()
result_cpu = vector_sum_cpu(vector)
end_time_cpu = time.time()

cpu_time = end_time_cpu - start_time_cpu
print(f"Сумма на CPU: {result_cpu}, Время выполнения на CPU: {cpu_time} секунд")

# Сложение на GPU
start_time_gpu = time.time()
result_gpu = vector_sum_gpu(vector)
end_time_gpu = time.time()

gpu_time = end_time_gpu - start_time_gpu
print(f"Сумма на GPU: {result_gpu}, Время выполнения на GPU: {gpu_time} секунд")    

Сумма на CPU: 49995.38013009982, Время выполнения на CPU: 0.005003452301025391 секунд
Сумма на GPU: 49995.38013009983, Время выполнения на GPU: 0.00499415397644043 секунд


In [18]:
vector_size = 1000000
vector = np.random.rand(vector_size)

# Сложение на CPU
start_time_cpu = time.time()
result_cpu = vector_sum_cpu(vector)
end_time_cpu = time.time()

cpu_time = end_time_cpu - start_time_cpu
print(f"Сумма на CPU: {result_cpu}, Время выполнения на CPU: {cpu_time} секунд")

# Сложение на GPU
start_time_gpu = time.time()
result_gpu = vector_sum_gpu(vector)
end_time_gpu = time.time()

gpu_time = end_time_gpu - start_time_gpu
print(f"Сумма на GPU: {result_gpu}, Время выполнения на GPU: {gpu_time} секунд")    

Сумма на CPU: 500320.66229143896, Время выполнения на CPU: 0.05040907859802246 секунд
Сумма на GPU: 500320.66229143133, Время выполнения на GPU: 0.00500035285949707 секунд


In [21]:
vector_size = 10000000
vector = np.random.rand(vector_size)

# Сложение на CPU
start_time_cpu = time.time()
result_cpu = vector_sum_cpu(vector)
end_time_cpu = time.time()

cpu_time = end_time_cpu - start_time_cpu
print(f"Сумма на CPU: {result_cpu}, Время выполнения на CPU: {cpu_time} секунд")

# Сложение на GPU
start_time_gpu = time.time()
result_gpu = vector_sum_gpu(vector)
end_time_gpu = time.time()

gpu_time = end_time_gpu - start_time_gpu
print(f"Сумма на GPU: {result_gpu}, Время выполнения на GPU: {gpu_time} секунд")    

Сумма на CPU: 4999503.079631773, Время выполнения на CPU: 0.500321626663208 секунд
Сумма на GPU: 4999503.079632387, Время выполнения на GPU: 0.09285140037536621 секунд


In [20]:
vector_size = 100000000
vector = np.random.rand(vector_size)

# Сложение на CPU
start_time_cpu = time.time()
result_cpu = vector_sum_cpu(vector)
end_time_cpu = time.time()

cpu_time = end_time_cpu - start_time_cpu
print(f"Сумма на CPU: {result_cpu}, Время выполнения на CPU: {cpu_time} секунд")

# Сложение на GPU
start_time_gpu = time.time()
result_gpu = vector_sum_gpu(vector)
end_time_gpu = time.time()

gpu_time = end_time_gpu - start_time_gpu
print(f"Сумма на GPU: {result_gpu}, Время выполнения на GPU: {gpu_time} секунд")    

Сумма на CPU: 50001617.07631788, Время выполнения на CPU: 5.042378187179565 секунд
Сумма на GPU: 50001617.07631542, Время выполнения на GPU: 0.6173648834228516 секунд


In [22]:
vector_size = 100000000
vector = np.random.rand(vector_size)

# Сложение на CPU
start_time_cpu = time.time()
result_cpu = vector_sum_cpu(vector)
end_time_cpu = time.time()

cpu_time = end_time_cpu - start_time_cpu
print(f"Сумма на CPU: {result_cpu}, Время выполнения на CPU: {cpu_time} секунд")

# Сложение на GPU
start_time_gpu = time.time()
result_gpu = vector_sum_gpu(vector)
end_time_gpu = time.time()

gpu_time = end_time_gpu - start_time_gpu
print(f"Сумма на GPU: {result_gpu}, Время выполнения на GPU: {gpu_time} секунд")    

Сумма на CPU: 49998562.984551504, Время выполнения на CPU: 4.891952991485596 секунд
Сумма на GPU: 49998562.98455838, Время выполнения на GPU: 0.4043457508087158 секунд
